In [1]:
import itertools
from copy import deepcopy

import ipysheet
from ipysheet.pandas_loader import to_dataframe

from util.convert import process_aspif
from util.explain import Literal, preprocess, explanation_graph, get_minimal_assumptions, negation_atoms
from util.run import solve

import networkx as nx

In [2]:
import clingo

In [3]:
import subprocess

## Input

In [4]:
rows,columns = 6,1

In [5]:
facts_sheet = ipysheet.sheet(rows=rows, columns=columns)

cells = []

for row in range(rows):
    cell_range = []
    for column in range(columns):
        cell = ipysheet.cell(row, column, "")
        cell_range.append(cell)
    cells.append(cell_range)

#cells[0][0] = ipysheet.cell(0,0, "a")
cells[4][0] = ipysheet.cell(4,0, "e")

facts_sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value=''), Cell(column_en…

In [6]:
df = to_dataframe(facts_sheet)
df

,A
0,
1,
2,
3,
4,e
5,


In [7]:
facts_list = list(itertools.chain(*df[df != ""].dropna().values.tolist()))
facts_list

['e']

## Programm

In [8]:
facts = '\n'.join(map(lambda f: f"{f}.", facts_list))
facts

'e.'

In [9]:
program = """
a :- k, not b.
k :- e, not b.
c :- a, b.

b :- not a.
c :- k.
f :- e, not k, not c.
"""
program

'\na :- k, not b.\nk :- e, not b.\nc :- a, b.\n\nb :- not a.\nc :- k.\nf :- e, not k, not c.\n'

## Answer Sets

In [10]:
answer_sets_clingo, models_clingo = solve((facts, program))

Answer  1: { b e f }.
Answer  2: { a c e k }.
SAT 2 


In [11]:
answer_sets_clingo

[{Function('b', [], True), Function('e', [], True), Function('f', [], True)},
 {Function('a', [], True),
  Function('c', [], True),
  Function('e', [], True),
  Function('k', [], True)}]

## Grounding

In [12]:
groundable_facts = list(map(lambda f: f"#external {f}.", facts_list))

with open('/tmp/asp_to_ground.lp', 'w') as file:
    file.write("% Facts:\n")
    file.writelines(groundable_facts)
    file.write("% Rules:\n")
    file.write(program)

In [13]:
output = subprocess.run(['gringo', '/tmp/asp_to_ground.lp'], stdout=subprocess.PIPE)


In [14]:
aspif = output.stdout.decode('utf-8')

In [15]:
program_dict = {}
literal_dict = {}

In [16]:
process_aspif(aspif, program_dict, literal_dict)

({e: [{'positive': [], 'negative': []}],
  b: [{'positive': [], 'negative': [a]}],
  k: [{'positive': [e], 'negative': [b]}],
  a: [{'positive': [k], 'negative': [b]}],
  c: [{'positive': [b, a], 'negative': []}, {'positive': [k], 'negative': []}],
  f: [{'positive': [e], 'negative': [c, k]}]},
 {1: e, 2: b, 4: k, 3: a, 5: c, 6: f})

In [17]:
answer_sets = [{Literal.literal_from_symbol(elem) for elem in answer_set} for answer_set in answer_sets_clingo]
answer_sets

[{b, e, f}, {a, c, e, k}]

In [18]:
cautious_consequence = answer_sets[0]
for answer_set in answer_sets[1:]:
    cautious_consequence.intersection_update(answer_set)
cautious_consequence

{e}

In [19]:
facts_literals = {Literal.literal_from_string(fact) for fact in facts_list}
facts_literals

{e}

In [20]:
derivable_dict = preprocess(program_dict, facts_literals, answer_sets[0])
derivable_dict

{e: [{'T'}],
 ~b: [],
 ~k: [],
 ~a: [frozenset({~k})],
 ~c: [frozenset({~b, ~k}), frozenset({~a, ~k})],
 ~f: []}

In [23]:
minimal_assumptions = get_minimal_assumptions(deepcopy(cautious_consequence), negation_atoms(program_dict), deepcopy(derivable_dict), answer_sets[0])
minimal_assumptions

{frozenset({frozenset({a, b, c, k})})}

In [22]:
graph = explanation_graph(Literal.literal_from_string('f'), deepcopy(derivable_dict), deepcopy(minimal_assumptions), deepcopy(answer_sets[0]))
assert graph is not None
pos = nx.nx_pydot.pydot_layout(graph, 'dot', Literal.literal_from_string('f'))
nx.draw(graph, pos=pos, with_labels=True)

TypeError: bad operand type for unary -: 'frozenset'